In [1]:
# ========== 第一步：安装依赖库（仅运行1次） ==========
!pip install pandas matplotlib seaborn -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
# ========== 第二步：导入库+配置环境（每次运行必跑） ==========
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")  # 忽略无关警告

# 配置中文字体，避免图表乱码
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

In [7]:
# ========== 第三步：开始业务代码（数据加载） ==========
# 1. 加载数据（替换为你的本地文件路径）
df = pd.read_csv("/Users/admin/Downloads/retail_sales_dataset.csv")

# 2. 数据基础探索（逐行运行，观察输出结果）
# ① 查看数据前5行、后5行（了解字段内容）
print("=== 数据前5行 ===")
print(df.head())
print("\n=== 数据后5行 ===")
print(df.tail())

# ② 查看数据基本信息（行数、列数、字段类型、缺失值数量）
print("\n=== 数据基本信息 ===")
print(df.info())  # 重点看：非空值数量（缺失值=总行数-非空值数）、字段类型（如sale_date是否为datetime）

# ③ 查看数值型字段统计信息（找异常值，如销量为负、单价异常）
print("\n=== 数值型字段统计 ===")
print(df.describe())  # 重点看quantity_sold（销量）、unit_price（单价）的min值（是否为负/0）

# ④ 查看分类字段唯一值（确认品类、门店数量是否合理）
print("\n=== 商品品类唯一值 ===")
print(df["Product Category"].unique())
print("\n=== 门店编号唯一值 ===")
print(df["Transaction ID"].unique())

# ⑤ 统计缺失值占比（更直观）
print("\n=== 各字段缺失值占比 ===")
missing_rate = (df.isnull().sum() / len(df)) * 100
print(missing_rate.round(2))  # 保留2位小数

=== 数据前5行 ===
   Transaction ID        Date Customer ID  Gender  Age Product Category  \
0               1  2023-11-24     CUST001    Male   34           Beauty   
1               2  2023-02-27     CUST002  Female   26         Clothing   
2               3  2023-01-13     CUST003    Male   50      Electronics   
3               4  2023-05-21     CUST004    Male   37         Clothing   
4               5  2023-05-06     CUST005    Male   30           Beauty   

   Quantity  Price per Unit  Total Amount  
0         3              50           150  
1         2             500          1000  
2         1              30            30  
3         1             500           500  
4         2              50           100  

=== 数据后5行 ===
     Transaction ID        Date Customer ID  Gender  Age Product Category  \
995             996  2023-05-16     CUST996    Male   62         Clothing   
996             997  2023-11-17     CUST997    Male   52           Beauty   
997             998  2023

In [8]:
# 1. 检查异常值（销量、单价）
print("=== 异常值检查 ===")
print("销量为负/0的记录数：", len(df[df["Quantity"] <= 0]))
print("单价为负/0的记录数：", len(df[df["Price per Unit"] <= 0]))

# 2. 处理异常值（如果有，删除异常记录）
df = df[df["Quantity"] > 0]  # 只保留销量>0的记录
df = df[df["Price per Unit"] > 0]  # 只保留单价>0的记录

# 3. 处理日期格式（转成datetime类型）
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")  # 转格式，失败的设为NaN
df = df.dropna(subset=["Date"])  # 若有转换失败的日期，删除对应行

# 4. 检查+处理重复值（按交易ID去重）
print("\n=== 重复值检查 ===")
print("重复交易ID的记录数：", df["Transaction ID"].duplicated().sum())
df = df.drop_duplicates(subset=["Transaction ID"], keep="first")  # 去重

# 清洗后的数据行数
print("\n清洗后数据总行数：", len(df))

=== 异常值检查 ===
销量为负/0的记录数： 0
单价为负/0的记录数： 0

=== 重复值检查 ===
重复交易ID的记录数： 0

清洗后数据总行数： 1000


In [10]:
# ========== 清洗后数据最终校验 ==========
print("===== 清洗后数据核心校验结果 =====")

# 1. 基础信息校验（行数、列数）
print(f"1. 清洗后数据总行数：{len(df)}")
print(f"2. 数据总列数：{len(df.columns)}")
print(f"3. 所有列名：{df.columns.tolist()}")

# 2. 缺失值最终校验（确保无缺失）
missing_rate = (df.isnull().sum() / len(df)) * 100
print("\n4. 各字段缺失值占比（目标：0%）：")
for col in df.columns:
    print(f"   - {col}：{missing_rate[col]:.2f}%")

# 3. 异常值最终校验（销量、单价均>0）
print("\n5. 异常值校验（目标：无异常）：")
print(f"   - 销量≤0的记录数：{len(df[df['Quantity'] <= 0])}")
print(f"   - 单价≤0的记录数：{len(df[df['Price per Unit'] <= 0])}")

# 4. 日期格式校验（确保是datetime类型）
print("\n6. 日期格式校验：")
print(f"   - Date字段类型：{df['Date'].dtype}（目标：datetime64[ns]）")
print(f"   - 日期转换失败的记录数：{len(df[df['Date'].isnull()])}")

# 5. 重复值校验（交易ID无重复）
print("\n7. 重复值校验：")
print(f"   - 重复交易ID记录数：{df['Transaction ID'].duplicated().sum()}（目标：0）")

# 6. 数值字段统计校验（确认范围合理）
print("\n8. 核心数值字段统计（参考）：")
print(df[['Quantity', 'Price per Unit', 'Total Amount']].describe().round(2))

===== 清洗后数据核心校验结果 =====
1. 清洗后数据总行数：1000
2. 数据总列数：9
3. 所有列名：['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']

4. 各字段缺失值占比（目标：0%）：
   - Transaction ID：0.00%
   - Date：0.00%
   - Customer ID：0.00%
   - Gender：0.00%
   - Age：0.00%
   - Product Category：0.00%
   - Quantity：0.00%
   - Price per Unit：0.00%
   - Total Amount：0.00%

5. 异常值校验（目标：无异常）：
   - 销量≤0的记录数：0
   - 单价≤0的记录数：0

6. 日期格式校验：
   - Date字段类型：datetime64[ns]（目标：datetime64[ns]）
   - 日期转换失败的记录数：0

7. 重复值校验：
   - 重复交易ID记录数：0（目标：0）

8. 核心数值字段统计（参考）：
       Quantity  Price per Unit  Total Amount
count   1000.00         1000.00        1000.0
mean       2.51          179.89         456.0
std        1.13          189.68         560.0
min        1.00           25.00          25.0
25%        1.00           30.00          60.0
50%        3.00           50.00         135.0
75%        4.00          300.00         900.0
max        4.00          500.00        2000.0


In [11]:
# ========== 清洗后数据保存 ==========
# 保存路径（Mac系统，可根据你的需求修改）
save_path = "/Users/admin/Downloads/retail_sales_cleaned.csv"

# 保存CSV文件（UTF-8编码避免中文乱码，不保存索引列）
df.to_csv(
    save_path,
    index=False,  # 不保存行索引（避免新增无用列）
    encoding="utf-8"  # 编码格式
)

# 验证保存是否成功
print(f"\n✅ 清洗后数据已保存至：{save_path}")
# 读取保存的文件，确认内容正常
df_check = pd.read_csv(save_path)
print(f"✅ 保存文件校验：读取到{len(df_check)}行数据，列名：{df_check.columns.tolist()}")


✅ 清洗后数据已保存至：/Users/admin/Downloads/retail_sales_cleaned.csv
✅ 保存文件校验：读取到1000行数据，列名：['Transaction ID', 'Date', 'Customer ID', 'Gender', 'Age', 'Product Category', 'Quantity', 'Price per Unit', 'Total Amount']
